In [3]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\Processsed_Data\\LTK_Invoice.csv")
df2 = pd.read_csv("C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\Processsed_Data\\SAFARICOM_Invoice.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\USER\\OneDrive\\KRA WORK\\2024 WS\\SLA EDA\\Processsed_Data\\LTK_Invoice.csv'

In [5]:
df.head()

,Invoice Date,Invoice No,Invoice Period,Link ID,Invoice Description,QRC (EX TAX),Excise (15%),VAT (16%),Invoice Total
0,2023-10-13,CRN-15473,01-Oct-2023 to 31-Dec-2023,26046,26046 National MPLS KRA MBITA TO VRF (INS-57),-48264.0,0,-7722.24,-55986.24
1,2023-10-06,116426,01-Oct-2023 to 31-Dec-2023,C-00159-0087,C-00159-0087 EPL Connection charge 80MBPS->SAM...,360000.0,0,57600.00,417600.00
2,2023-10-06,116426,01-Oct-2023 to 31-Dec-2023,1750,01750 National MPLS KRA - TIMES TOWERS TO VRF,810000.0,0,129600.00,939600.00
3,2023-10-06,116426,01-Oct-2023 to 31-Dec-2023,5611,05611 National MPLS Msa link vlan 460 customs,61927.5,0,9908.40,71835.90
4,2023-10-06,116426,01-Oct-2023 to 31-Dec-2023,20103,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,68850.0,0,11016.00,79866.00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Invoice Date         26 non-null     object 
 1   Invoice No           26 non-null     object 
 2   Invoice Period       26 non-null     object 
 3   Link ID              26 non-null     object 
 4   Invoice Description  26 non-null     object 
 5   QRC (EX TAX)         26 non-null     float64
 6   Excise (15%)         26 non-null     int64  
 7   VAT (16%)            26 non-null     float64
 8   Invoice Total        26 non-null     float64
dtypes: float64(3), int64(1), object(5)
memory usage: 2.0+ KB


In [6]:
df2.head()

NameError: name 'df2' is not defined

In [5]:
df2.head()

NameError: name 'df2' is not defined

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   No.                          59 non-null     object 
 1   Invoice Date                 58 non-null     object 
 2   Invoice Ref.                 58 non-null     object 
 3   Invoice Period               58 non-null     object 
 4   Link ID                      58 non-null     float64
 5   Invoice Description          58 non-null     object 
 6   Net billed(Exc. Taxes)       59 non-null     float64
 7   VAT                          59 non-null     float64
 8   Excise                       59 non-null     float64
 9   Monthly Total Billed         59 non-null     float64
 10  Period(Months)               58 non-null     float64
 11  Total Billed for the Period  58 non-null     float64
 12  Amount Including Excise      59 non-null     float64
dtypes: float64(8), object(

In [8]:
df2.tail()

,No.,Invoice Date,Invoice Ref.,Invoice Period,Link ID,Invoice Description,Net billed(Exc. Taxes),VAT,Excise,Monthly Total Billed,Period(Months),Total Billed for the Period,Amount Including Excise
54,55,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95096538.0,KRA BUSIA-WIMAX,7.000830e+04,12881.527200,10501.24500,9.339107e+04,3.0,280173.2166,8.050954e+04
55,56,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95096687.0,KRA CUSTOMS OFFICE- NAIROBI-NAMANGA WIMAX,6.999750e+04,12879.540000,10499.62500,9.337666e+04,3.0,280129.9950,8.049712e+04
56,57,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95096716.0,KRA ISEBANIA-WIMAX,7.368510e+04,13558.058400,11052.76500,9.829592e+04,3.0,294887.7702,8.473787e+04
57,58,2023-10-01,B1-10096515502,1st Oct to 31st Dec 2023,95096873.0,KRA LUNGA LUNGA WIMAX,6.999750e+04,12879.540000,10499.62500,9.337666e+04,3.0,280129.9950,8.049712e+04
58,Total,NaN,NaN,NaN,NaN,NaN,2.815170e+06,517991.341566,422275.55019,3.755437e+06,NaN,NaN,3.237446e+06


In [2]:
# Dropping rows with null values in all the fields
df2.dropna(how='all', inplace = True)

#dropping the total row
df2.drop(df2[df2["No."] == "Total"].index, inplace= True)

##Function to help pick the columns
def pick_Invoice_no_column_name(df):
    # Loop through the fixed substrings
    for substring in ['inveoice no', 'ref']:
        # Check if any column contains the substring
        matching_columns = [col for col in df.columns if substring.lower() in col.lower()]
        # If a matching column is found, return its name
        if matching_columns:
            return matching_columns[0]
        
def QRC_Finder(df):
    max_value = None
    max_col = None
    for substring in ['QRC', 'Total', 'Amount']:
        #Checking for the matching amount related columns
        fields = [col for col in df.columns if substring.lower() in col.lower()]
    for field in fields:
        # Check if the column exists and has valid values
        if field in df.columns and not df[field].isnull().all():
            # Finding the maximum value in the first row of the column
            first_row_value = df.at[0,field]
            if max_value is None or first_row_value > max_value:
                max_value = first_row_value
                max_col = field
    return max_col



# Dropping unwanted Columns
columns_wanted = {"Date": "Invoice Date",
                  "ID": "Link ID",
                   "Period": "Invoice Period",
                    "Descriprion": "Invoice Description",
                     "QRC":QRC_Finder(df2),
                      "Reference/ no": pick_Invoice_no_column_name(df2)}

df_selected = df2.loc[:, [value for value in columns_wanted.values()]]



NameError: name 'df2' is not defined

In [35]:
df_selected.head()

,Invoice Date,Link ID,Invoice Period,Invoice Description,Amount Including Excise,Invoice Ref.
0,2023-10-01,95001135.0,1st Oct to 31st Dec 2023,CUSTOMS OFFICE MALINDI-WIMAX,69007.36,B1-10096515502
1,2023-10-01,95001574.0,1st Oct to 31st Dec 2023,KRA LOITOKTOK BORDER WIMAX,69007.36,B1-10096515502
2,2023-10-01,95001628.0,1st Oct to 31st Dec 2023,KRA KRATI MOMBASA WIMAX,69007.36,B1-10096515502
3,2023-10-01,95004139.0,1st Oct to 31st Dec 2023,KRA - MANDERA,51746.78,B1-10096515502
4,2023-10-01,95004140.0,1st Oct to 31st Dec 2023,KRA - Moyale,69007.36,B1-10096515502


In [29]:
columns_wanted = {"Date": "Invoice Date",
                  "ID": "Link ID",
                   "Period": "Invoice Period",
                    "Descriprion": "Invoice Description",
                     "QRC":"QRC_Finder(df2)",
                      "Reference/ no": "pick_Invoice_no_column_name(df2)"}
list(columns_wanted.values())

['Invoice Date',
 'Link ID',
 'Invoice Period',
 'Invoice Description',
 'QRC_Finder(df2)',
 'pick_Invoice_no_column_name(df2)']

In [24]:
not df2['Amount Including Excise'].isnull().all()


True

In [25]:
df2.at[0,'Invoice Date']

'2023-10-01'

In [21]:
def QRC_Finder(df):
    max_value = None
    max_column = None
    
    for substring in ['QRC', 'Total', 'Amount']:
        # Checking for the matching amount related columns
        fields = [col for col in df.columns if substring.lower() in col.lower()]
    
    for field in fields:
        # Check if the column exists and has valid values
        if field in df.columns and not df[field].isnull().all():
            # Find the maximum value in the first row of the column
            first_row_value = df.at[0, field]
            if max_value is None or first_row_value > max_value:
                max_value = first_row_value
                max_column = field
    
    return max_column

# Example usage
df = pd.DataFrame({'Total Amount': [100, 150, 200], 'Invoice Amount': [120, 140, 180]})
largest_column = QRC_Finder(df)
print("Column with largest value:", largest_column)


Column with largest value: Invoice Amount


In [ ]:
df2.tail()

In [18]:
import pandas as pd

def pick_column(df, substrings):
    # Initialize an empty list to store the filtered columns
    wanted_columns = []
    
    # Loop through each substring and filter columns
    for substring in substrings:
        # Filter columns containing the current substring
        columns = df.columns[df.columns.str.contains(substring, case=False)]
        # Extend the list of wanted columns with the filtered columns
        wanted_columns.extend(columns)
    
    # Remove duplicates and return the final list of wanted columns
    return list(set(wanted_columns))

# Example usage
df = pd.DataFrame({'Invoice Ref': [1, 2, 3], 'Invoice Kenya.': [4, 5, 6]})
wanted_columns = pick_column(df, ['no.', 'ref'])
print(wanted_columns)


['Invoice Ref']


In [20]:
import pandas as pd

def pick_column_name(df):
    # Loop through the fixed substrings
    for substring in ['no.', 'ref']:
        # Check if any column contains the substring
        matching_columns = [col for col in df.columns if substring.lower() in col.lower()]
        # If a matching column is found, return its name
        if matching_columns:
            return matching_columns[0]

# Example usage
df = pd.DataFrame({'Invoice Ref': [1, 2, 3], 'Invoice kenya.': [4, 5, 6]})
wanted_column = pick_column_name(df)
print(wanted_column)


Invoice Ref


In [1]:
import pandas as pd

class InvoiceCleaner:
    def __init__(self, df):
        self.df = df
        
    def drop_null_rows(self):
        self.df.dropna(how='all', inplace=True)
        
    def drop_total_row(self):
        if "No." in self.df.columns:
            self.df.drop(self.df[self.df["No."] == "Total"].index, inplace=True)
        
        
    def pick_invoice_no_column_name(self):
        for substring in ['invoice no', 'ref']:
            matching_columns = [col for col in self.df.columns if substring.lower() in col.lower()]
            if matching_columns:
                return matching_columns[0]
                
    def QRC_Finder(self):
        max_value = None
        max_col = None
        for substring in ['QRC', 'Total', 'Amount']:
            fields = [col for col in self.df.columns if substring.lower() in col.lower()]
            for field in fields:
                if field in self.df.columns and not self.df[field].isnull().all():
                    first_row_value = self.df.at[1, field]
                    if max_value is None or first_row_value > max_value:
                        max_value = first_row_value
                        max_col = field
        return max_col
    
    def clean_invoice_data(self):
        self.drop_null_rows()
        self.drop_total_row()
        
        columns_wanted = {
            "Date": "Invoice Date",
            "ID": "Link ID",
            "Period": "Invoice Period",
            "Descriprion": "Invoice Description",
            "QRC": self.QRC_Finder(),
            "Reference/ no": self.pick_invoice_no_column_name()
        }

        self.df = self.df.loc[:, [value for value in columns_wanted.values()]]


In [62]:
# Assuming df2 is your DataFrame
cleaner = InvoiceCleaner(df2)
cleaner.clean_invoice_data()
cleaned_df = cleaner.df
cleaned_df.head()


,Invoice Date,Link ID,Invoice Period,Invoice Description,Total Billed for the Period,Invoice Ref.
0,2023-10-01,95001135.0,1st Oct to 31st Dec 2023,CUSTOMS OFFICE MALINDI-WIMAX,240145.6128,B1-10096515502
1,2023-10-01,95001574.0,1st Oct to 31st Dec 2023,KRA LOITOKTOK BORDER WIMAX,240145.6128,B1-10096515502
2,2023-10-01,95001628.0,1st Oct to 31st Dec 2023,KRA KRATI MOMBASA WIMAX,240145.6128,B1-10096515502
3,2023-10-01,95004139.0,1st Oct to 31st Dec 2023,KRA - MANDERA,180078.7944,B1-10096515502
4,2023-10-01,95004140.0,1st Oct to 31st Dec 2023,KRA - Moyale,240145.6128,B1-10096515502


In [63]:
# Assuming df2 is your DataFrame
cleaner = InvoiceCleaner(df)
cleaner.clean_invoice_data()
cleaned_df = cleaner.df
cleaned_df.head()

,Invoice Date,Link ID,Invoice Period,Invoice Description,Invoice Total,Invoice No
0,2023-10-13,26046,01-Oct-2023 to 31-Dec-2023,26046 National MPLS KRA MBITA TO VRF (INS-57),-55986.24,CRN-15473
1,2023-10-06,C-00159-0087,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,417600.00,116426
2,2023-10-06,1750,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,939600.00,116426
3,2023-10-06,5611,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,71835.90,116426
4,2023-10-06,20103,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,79866.00,116426


In [64]:
# Assuming df2 is your DataFrame
cleaner = InvoiceCleaner(df)
cleaner.clean_invoice_data()

# Access the cleaned data
cleaned_data = cleaner.df
cleaned_df.head()

,Invoice Date,Link ID,Invoice Period,Invoice Description,Invoice Total,Invoice No
0,2023-10-13,26046,01-Oct-2023 to 31-Dec-2023,26046 National MPLS KRA MBITA TO VRF (INS-57),-55986.24,CRN-15473
1,2023-10-06,C-00159-0087,01-Oct-2023 to 31-Dec-2023,C-00159-0087 EPL Connection charge 80MBPS->SAM...,417600.00,116426
2,2023-10-06,1750,01-Oct-2023 to 31-Dec-2023,01750 National MPLS KRA - TIMES TOWERS TO VRF,939600.00,116426
3,2023-10-06,5611,01-Oct-2023 to 31-Dec-2023,05611 National MPLS Msa link vlan 460 customs,71835.90,116426
4,2023-10-06,20103,01-Oct-2023 to 31-Dec-2023,20103 Leased_Circuits_Naticnal KRA WILSON AIRP...,79866.00,116426


In [43]:
cleaned_data.head()

,Invoice Date,Link ID,Invoice Period,Invoice Description,Total Billed for the Period,Invoice Ref.
0,2023-10-01,95001135.0,1st Oct to 31st Dec 2023,CUSTOMS OFFICE MALINDI-WIMAX,240145.6128,B1-10096515502
1,2023-10-01,95001574.0,1st Oct to 31st Dec 2023,KRA LOITOKTOK BORDER WIMAX,240145.6128,B1-10096515502
2,2023-10-01,95001628.0,1st Oct to 31st Dec 2023,KRA KRATI MOMBASA WIMAX,240145.6128,B1-10096515502
3,2023-10-01,95004139.0,1st Oct to 31st Dec 2023,KRA - MANDERA,180078.7944,B1-10096515502
4,2023-10-01,95004140.0,1st Oct to 31st Dec 2023,KRA - Moyale,240145.6128,B1-10096515502
